<a href="https://colab.research.google.com/github/timvdstap/misc-code-snippets/blob/main/Querying_Biological_DataServices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following is an Rmarkdown file based on Stephen Formel's (USGS) blogpost: https://www.gbif.us/post/2024/searching-with-aphiaids/. My goal here is to use this approach and code to show the occurrences of basking sharks (Cetorhinus maximus) in Canadian waters. Let's first install and load the relevant packages:

In [ ]:
list_of_packages <- c("dplyr", "mregions2", "sf", "wk", "worrms", "httr2", "tidyr", "ggplot2", "rnaturalearth",
                      "rfishbase", "robis", "rgbif", "stringr", "ggtext")
new_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(new_packages)) install.packages(new_packages)

library(dplyr)
library(mregions2)
library(sf)
library(wk)
library(worrms)
library(httr2)
library(tidyr)
library(ggplot2)
library(rnaturalearth)
library(rfishbase)
library(robis)
library(rgbif)
library(stringr)
library(ggtext)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘mregions2’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
also installing the dependencies ‘geometries’, ‘jsonify’, ‘rapidjsonr’, ‘sfheaders’, ‘xfun’, ‘proxy’, ‘triebeard’, ‘geojsonsf’, ‘leafpop’, ‘satellite’, ‘servr’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘e1071’, ‘urltools’, ‘httpcode’, ‘crosstalk’, ‘htmlwidgets’, ‘leaflet.providers’, ‘png’, ‘raster’, ‘sp’, ‘leafem’, ‘leaflet.extras’, ‘leafpm’, ‘mapview’, ‘miniUI’, ‘shiny’, ‘plyr’, ‘markdown’, ‘jpeg’, ‘classInt’, ‘Rcpp’, ‘s2’, ‘units’, ‘crul’, ‘terra’, ‘duckdb’, ‘contentid’, ‘leaflet’, ‘httpcache’, ‘mapedit’, ‘oai’, ‘lazyeval’, ‘gridtext’




You need the following code to install and load the mregions2 package

In [ ]:
install.packages("devtools")
devtools::install_github("lifewatch/mregions2")

Next, let's have a look at the basking shark distribution within the Canadian EEZ

In [ ]:
AphiaID <- worrms::wm_name2id(name = "Cetorhinus maximus")

fishbaseID <- httr2::request(base_url = "https://www.marinespecies.org/rest/AphiaExternalIDByAphiaID/") %>%
  httr2::req_url_path_append(AphiaID) %>%
  httr2::req_url_query(`type` = 'fishbase') %>%
  httr2::req_perform() %>%
  httr2::resp_body_json() %>%
  unlist()

sourceID <- paste0('urn:lsid:marinespecies.org:taxname:', AphiaID)

response <- httr2::request(base_url = 'https://api.gbif.org/v1/species') %>%
  httr2::req_url_query(`datasetKey` = '2d59e5db-57ad-41ff-97d6-11f5fb264527',
                       `sourceID` = 'urn:lsid:marinespecies.org:taxname:105837') %>%
  httr2::req_perform() %>%
  resp_body_json()

GBIF_backboneID <- response$results[[1]]$nubKey